In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install tabpfn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing instal

In [6]:
import pandas as pd
import joblib
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tabpfn import TabPFNClassifier

# Load dataset
df = pd.read_csv("/kaggle/input/cricket-dataset/final_odi_matches_with_full_weather2.csv")

# Clean and engineer
df = df.dropna(subset=['Match Winner', 'Team1 Name', 'Team2 Name', 'Avg_Temp_C'])
df['Avg_Temp_C'] = df['Avg_Temp_C'].replace(-99, np.nan)
df = df.dropna(subset=['Avg_Temp_C'])

df.rename(columns={
    'Team1 Name': 'Team1',
    'Team2 Name': 'Team2',
    'Match Winner': 'Winner',
    'Toss Winner': 'TossWinner',
    'Match Venue (Country)': 'VenueCountry'
}, inplace=True)

df['home_advantage'] = df.apply(
    lambda row: 1 if pd.notna(row['VenueCountry']) and row['VenueCountry'].lower() in row['Team1'].lower() else 0,
    axis=1
)
df['won_toss'] = df.apply(lambda row: 1 if row['TossWinner'] == row['Team1'] else 0, axis=1)

team_rank = {
    'India': 1, 'New Zealand': 2, 'Australia': 3, 'Sri Lanka': 4,
    'Pakistan': 5, 'South Africa': 6, 'Afghanistan': 7,
    'England': 8, 'West Indies': 9, 'Bangladesh': 10
}
df['team1_rank'] = df['Team1'].map(team_rank)
df['team2_rank'] = df['Team2'].map(team_rank)
df = df.dropna(subset=['team1_rank', 'team2_rank'])
df['rank_diff'] = df['team2_rank'] - df['team1_rank']
df['target'] = df.apply(lambda row: 1 if row['Winner'] == row['Team1'] else 0, axis=1)

# Final features
features = ['Avg_Temp_C', 'home_advantage', 'won_toss', 'team1_rank', 'team2_rank', 'rank_diff']
X = df[features]
y = df['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# Train TabPFN
tabpfn_clf = TabPFNClassifier(device="cuda" if torch.cuda.is_available() else "cpu")
tabpfn_clf.fit(X_train, y_train)

# Predict & Evaluate
y_pred = tabpfn_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
# Train TabPFN
tabpfn_clf = TabPFNClassifier(device="cuda" if torch.cuda.is_available() else "cpu")
tabpfn_clf.fit(X_train, y_train)

# Save model to disk
joblib.dump(tabpfn_clf, "tabpfn_model.pkl")
print("✅ Model saved as tabpfn_model.pkl")


Accuracy: 0.6474056603773585
              precision    recall  f1-score   support

           0       0.66      0.60      0.63       423
           1       0.63      0.70      0.67       425

    accuracy                           0.65       848
   macro avg       0.65      0.65      0.65       848
weighted avg       0.65      0.65      0.65       848

✅ Model saved as tabpfn_model.pkl
